<a href="https://colab.research.google.com/github/tayfununal/2-Hidden-Layer-Neural-Networks/blob/master/baski_Maliyet_Fonksiyonu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playground-data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plygdata as pg
import json
import requests

In [ ]:
Datas = json.loads(requests.get("https://raw.githubusercontent.com/tayfununal/2-Hidden-Layer-Neural-Networks/master/produced_XOR_Datas.json").text)

In [ ]:
df = pd.DataFrame(Datas["1"])
df

In [ ]:
X = np.array(df[[0,1]]).T
Y = np.array(df[[2]]).T

In [ ]:
print("Shape of X_train:", X.shape,
      "\nShape of y_train:", Y.shape)

In [ ]:
def initialization_parameters(x, y, num_node):
  W1 = np.random.randn(num_node, x.shape[0])
  b1 = np.zeros((num_node,1))

  W2 = np.random.randn(y.shape[0],num_node)
  b2 = np.zeros((y.shape[0],1))

  assert W1.shape == (num_node,x.shape[0])
  assert b1.shape == (num_node, 1)

  assert W2.shape == (y.shape[0], num_node)
  assert b2.shape == (y.shape[0], 1)

  parameters = {'W1':W1,
                'b1':b1,
                'W2':W2,
                'b2':b2}
  return parameters

def sigmoid(x):
  return 1/(1+np.exp(-x))

def relu(x):
  return np.maximum(0,x)

def reluDerivative(x):
  x[x<=0] = 0
  x[x>0] = 1
  return x

def forward_prop(x,parameters):
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  
  Z1 = np.dot(W1, x) + b1
  A1 = relu(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = sigmoid(Z2)

  assert (A2.shape == (1, x.shape[1]))
  cache = {
      'Z1' : Z1,
      'A1' : A1,
      'Z2' : Z2,
      'A2' : A2
  }
  return A2, cache

def backward_prop(x, y, parameters, baski_neuron, cache, learning_rate = 0.1):
  m = y.shape[1]
  
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']

  A1 = cache['A1']
  A2 = cache['A2']
  Z1 = cache['Z1']
  
  # Calculations of backward propagation: dW1, db1, dW2, db2
  baski_W1 = np.zeros((baski_neuron,2))
  baski_W1[baski_neuron-1,0] = gama * W1[baski_neuron-1,0]
  baski_W1[baski_neuron - 1, 1] = gama * W1[baski_neuron-1,1]

  baski_W2 = np.zeros((1,baski_neuron))
  baski_W2[0,baski_neuron-1] = gama * W2[0,baski_neuron-1]

  baski_b1=np.zeros((baski_neuron,1))
  baski_b1[baski_neuron-1,0] = gama * b1[baski_neuron-1,0]

  dZ2 = A2 - Y
  dW2 = (1.0 / m) * (np.dot(dZ2, A1.T) + baski_W2)
  db2 = (1.0 / m) * np.sum(dZ2, axis=1, keepdims=True)
  dZ1 = np.multiply(np.dot(W2.T, dZ2), reluDerivative(Z1))
  dW1 = (1.0 / m) * (np.dot(dZ1, X.T) + baski_W1)
  db1 = (1.0 / m) * (np.sum(dZ1, axis=1, keepdims=True) + baski_b1)

  # Updating parameters
  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2
  
  parameters = {'W1':W1,
                'b1':b1,
                'W2':W2,
                'b2':b2}
  return parameters

def cross_entropy_cost(y, A2, baski_neuron):
  m = y.shape[1]
  
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']

  cross_entropy = np.multiply(np.log(A2 + 1e-15), Y) + np.multiply((1 - Y), np.log(1 - A2 + 1e-15))
  cost = ( 1.0 / m) * ( - np.sum(cross_entropy) + (gama / 2.0) * (W1[baski_neuron-1,0]**2 + W1[baski_neuron-1,1]**2 + W2[0,baski_neuron-1]**2 + b1[baski_neuron-1,0]**2))
  
  # Squeezing to avoid unnecessary dimensions 
  cost = np.squeeze(cost) 
  return cost

In [ ]:
def nn_model(x, y, parameters, baski_neuron, number_of_iter = 1000):
  cost_value = {}
  A2, cache = forward_prop(x, parameters)
  cost = cross_entropy_cost(y, A2, baski_neuron)
  cost_value[1] = cost

  for i in range(1,number_of_iter):
    parameters = backward_prop(x, y, parameters, baski_neuron, cache, learning_rate=0.6)
    A2, cache = forward_prop(x, parameters)
    cost = cross_entropy_cost(y, A2, baski_neuron)
    
    #cost_value[i+1] = cost
    
    if (i+1) % 100 == 0:
     cost_value[i+1] = cost
     print("%i."%(i+1),cost)
    
  return cost_value, parameters

In [ ]:
np.random.seed(0)
gama = 5

parameters = initialization_parameters(X, Y, 46)
cost_value, parameters = nn_model(X, Y, parameters, baski_neuron = 46, number_of_iter = 1000)
  
"""
cost = []
for i in range(1,101):
  parameters = initialization_parameters(X, Y, i)
  cost_value, parameters = nn_model(X, Y, parameters, baski_neuron = i, number_of_iter = 1000)
  cost.append(cost_value[1000])
cost
"""

In [ ]:
parameters

In [ ]:
x = [i for i in range(1,101)]

plt.plot(x,cost)

for i in range(0,100):
  if cost[i]==0.007841210740944437:
    print(i)
  

In [ ]:
cost[48]